In [19]:
import numpy as np, os
from sklearn.neighbors import KDTree

In [31]:
BINS = 'Bins/'

IN_END = '/in.npy'

A, B = 148, 275 # posições dos coeficientes da onda inteira

def MkDirs(t):
    try:
        os.makedirs(t)
    except:
        pass

def GetBinOff(age):
    age = min(max(age, 17), 91)
    age -= 17
    return int(age) // 5

def GetPath(i):
    a = 17 + i * 5
    b = a + 4
    path = str(a) + '-' + str(b)
    return path

In [3]:
x = np.load('preprocessed0.npy')

In [4]:
x.shape

(40021, 278)

In [5]:
x[:,1]

array([78., 78., 78., ..., 80., 80., 80.])

In [6]:
for i in range(17, 91, 5):
    MkDirs(BINS + str(i) + '-' + str(i + 4))

In [7]:
x[333].shape

(278,)

In [26]:
l = [[] for i in range(15)]

for i in range(len(x)):
    row = x[i]
    age = row[1]
    off = GetBinOff(age)
    l[off].append(row)

ins = [np.array(i) for i in l]

In [37]:
for i in range(15):
    path = GetPath(i)
    np.save(BINS + path + IN_END, ins[i])

# KNN

In [53]:
for i in range(12): # último 72-76 pq 87-91 tem mto pouco
    dataDst = np.load(BINS + GetPath(i + 2) + IN_END) # 10 anos à frente
    ondas = []
    mappings = []
    for rowOff in range(len(dataDst)):
        row = dataDst[rowOff]
        if np.all(row[2:8] == 0): # labels
            c = row[A:B]
            ondas.append(c)
            mappings.append(rowOff)
    ondas = np.array(ondas)
    tree = KDTree(ondas)

    outStart = BINS + GetPath(i)
    dataSrc = np.load(outStart + IN_END)
    out = []
    for row in dataSrc:
        onda = row[A:B]
        dist, ind = tree.query(onda.reshape(1, -1))
        gIdx = dist.argmin()
        idxClosestOnda = ind[gIdx][0]
        orgRow = mappings[idxClosestOnda]
        out.append(dataDst[orgRow])
    np.save(outStart + '/out.npy', out)

# Teste

In [11]:
BINS_TEST = 'Bins/Test/'

In [12]:
for i in range(17, 91, 5):
    MkDirs(BINS_TEST + str(i) + '-' + str(i + 4))

In [16]:
x = np.load('test.npy')

In [17]:
l = [[] for i in range(15)]

for i in range(len(x)):
    row = x[i]
    age = row[0]
    off = GetBinOff(age)
    l[off].append(row)

ins = [np.array(i) for i in l]

for i in range(15):
    path = GetPath(i)
    np.save(BINS_TEST + path + IN_END, ins[i])

In [34]:
for i in range(12): # último 72-76 pq 87-91 tem mto pouco
    dataDst = np.load(BINS + GetPath(i + 2) + IN_END) # 10 anos à frente
    ondas = []
    mappings = []
    for rowOff in range(len(dataDst)):
        row = dataDst[rowOff]
        if np.all(row[2:8] == 0): # labels
            c = row[A:B]
            ondas.append(c)
            mappings.append(rowOff)
    ondas = np.array(ondas)
    tree = KDTree(ondas)

    outStart = BINS_TEST + GetPath(i)
    dataSrc = np.load(outStart + IN_END)
    out = []
    for row in dataSrc:
        onda = row[A-1:B-1]
        dist, ind = tree.query(onda.reshape(1, -1))
        gIdx = dist.argmin()
        idxClosestOnda = ind[gIdx][0]
        orgRow = mappings[idxClosestOnda]
        out.append(dataDst[orgRow])
    np.save(outStart + '/out.npy', out)